In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt


raw_results = pd.read_csv('../data/results.csv')
raw_races = pd.read_csv('../data/races.csv')
raw_drivers = pd.read_csv('../data/drivers.csv')
raw_circuits = pd.read_csv('../data/circuits.csv')
raw_constructor = pd.read_csv('../data/constructors.csv')
raw_status = pd.read_csv('../data/status.csv')
raw_constructor_standings = pd.read_csv('../data/constructor_standings.csv')
raw_qualify = pd.read_csv('../data/qualifying.csv')

#print(f'raw_results: {raw_results.shape}')
#print(f'raw_drivers: {raw_drivers.shape}')
#print(f'raw_circuits: {raw_circuits.shape}')
#print(f'raw_races: {raw_races.shape}')
#print(f'raw_constructor: {raw_constructor.shape}')
#print(f'raw_status: {raw_status.shape}')
#print(f'raw_qualify: {raw_qualify.shape}')

In [6]:
# remove unwated cols
raw_results = raw_results.drop(columns=['position', 'milliseconds'])
raw_races = raw_races.drop(columns=['url'])
raw_drivers = raw_drivers.drop(columns=['url'])
raw_circuits = raw_circuits.drop(columns=['url'])
raw_constructor = raw_constructor.drop(columns=['url'])
raw_constructor_standings = raw_constructor_standings.drop(columns=['positionText'])
#raw_status = raw_status.drop()
#raw_qualify = raw_qualify.drop()

In [7]:
#rename some cols before merging since it gets confusing what some cols are referring to and merge.suffix is inadequate
# don't rename the Id column
raw_circuits.columns = raw_circuits.columns.map(lambda x: str(x) + '_cir' if str(x)[-2:]!='Id' else str(x))
raw_results.columns = raw_results.columns.map(lambda x: str(x) + '_result' if str(x)[-2:]!='Id' else str(x))
raw_drivers.columns = raw_drivers.columns.map(lambda x: str(x) + '_driver' if str(x)[-2:]!='Id' else str(x))
raw_races.columns = raw_races.columns.map(lambda x: str(x) + '_race' if str(x)[-2:]!='Id' else str(x))
raw_constructor.columns = raw_constructor.columns.map(lambda x: str(x) + '_constr' if str(x)[-2:]!='Id' else str(x))
#raw_status.columns = raw_status.columns.map(lambda x: str(x) + '_stat' if str(x)[-2:]!='Id' else str(x))
raw_qualify.columns = raw_qualify.columns.map(lambda x: str(x) + '_qual' if str(x)[-2:]!='Id' else str(x))


In [8]:
#LEFT JOIN Results on Drivers
mg_results_driver = pd.merge(raw_results, raw_drivers, on='driverId', how='left', suffixes=('', '_driver'))
print(f'mg_results_driver: {mg_results_driver.shape}')
#LEFT JOIN Races on Circuits
mg_race_circuit = pd.merge(raw_races, raw_circuits, on='circuitId', how='left', suffixes=('_race', '_circuit'))
#Bring the two DFs from above together with LEFT JOIN Results/Drivers on Races/Circuits
mg_res_dr_rac_cir = pd.merge(mg_results_driver, mg_race_circuit, on='raceId', how='left', suffixes=('', '_race'))
#LEFT JOIN Above on Constructors
mg_res_dr_rac_cir_const = pd.merge(mg_res_dr_rac_cir, raw_constructor, on='constructorId', how='left', suffixes=('', '_constr'))
#LEFT JOIN Above on Atatus
df_all = pd.merge(mg_res_dr_rac_cir_const, raw_status, on='statusId', how='left')

#Check sapes to see if they look right and numer of rows matches between raw_results and df_all
print(f'mg_results_driver: {mg_results_driver.shape}')
print(f'mg_race_circuit: {mg_race_circuit.shape}')
print(f'mg_res_dr_rac_cir: {mg_res_dr_rac_cir.shape}')
print(f'mg_res_dr_rac_cir_const: {mg_res_dr_rac_cir_const.shape}')
print(f'df_all: {df_all.shape}')

mg_results_driver: (24620, 23)
mg_results_driver: (24620, 23)
mg_race_circuit: (1040, 14)
mg_res_dr_rac_cir: (24620, 36)
mg_res_dr_rac_cir_const: (24620, 39)
df_all: (24620, 40)


In [9]:
df_all.corr()

,resultId,raceId,driverId,constructorId,grid_result,positionOrder_result,points_result,laps_result,statusId,year_race,round_race,circuitId,lat_cir,lng_cir
resultId,1.000000,0.960034,0.697917,0.442897,-0.008497,-0.045180,0.204981,0.095441,0.014923,-0.137026,-0.022724,0.273415,0.022699,-0.021064
raceId,0.960034,1.000000,0.719324,0.438718,-0.012323,-0.038292,0.185081,0.088417,0.034501,-0.180031,-0.035727,0.294864,0.033902,-0.050678
driverId,0.697917,0.719324,1.000000,0.330241,0.083818,0.022516,-0.045099,0.093177,0.042692,-0.193426,-0.077219,0.199637,0.038850,-0.044227
constructorId,0.442897,0.438718,0.330241,1.000000,0.111464,0.094723,-0.059881,0.012358,0.125776,-0.327704,-0.131044,0.167654,0.058988,-0.077240
grid_result,-0.008497,-0.012323,0.083818,0.111464,1.000000,0.141802,-0.362405,0.081091,-0.132244,0.004859,0.017414,-0.001339,-0.006656,-0.011963
positionOrder_result,-0.045180,-0.038292,0.022516,0.094723,0.141802,1.000000,-0.566586,-0.656489,0.527974,-0.077603,-0.007340,0.004988,0.023610,-0.049611
points_result,0.204981,0.185081,-0.045099,-0.059881,-0.362405,-0.566586,1.000000,0.254507,-0.268427,0.253450,0.078317,-0.026738,-0.034314,0.087325
laps_result,0.095441,0.088417,0.093177,0.012358,0.081091,-0.656489,0.254507,1.000000,-0.354924,0.031481,-0.037948,0.010434,-0.035448,-0.098937
statusId,0.014923,0.034501,0.042692,0.125776,-0.132244,0.527974,-0.268427,-0.354924,1.000000,-0.206545,-0.083035,0.057604,0.033152,-0.087246
year_race,-0.137026,-0.180031,-0.193426,-0.327704,0.004859,-0.077603,0.253450,0.031481,-0.206545,1.000000,0.319338,-0.308759,-0.138805,0.298349


In [14]:
# deal with nulls - 
# or with this data '\N' values that were used to stand for null, which appear to be only in number cols
#df_all.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=[-1,-1], regex=True, inplace=True)
#df_all.replace(r'\\N',  -1, regex=True, inplace=True)
replace_val = -1
for col_name in df_all.columns:
    if df_all[col_name].dtype in ['int32', 'int64','float32', 'float64']:
        df_all[col_name].replace(r'\\N',  replace_val, regex=True, inplace=True)
    else:
        df_all[col_name].replace(r'\\N',  str(replace_val), regex=True, inplace=True)

In [15]:
print(f'time_result rows with newline escape: {df_all["time_result"][5:10]}')

time_result rows with newline escape: 5    -1
6    -1
7    -1
8    -1
9    -1
Name: time_result, dtype: object


In [16]:
# save to .csv
df_all.to_csv('data/df_all.csv')

In [17]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24620 entries, 0 to 24619
Data columns (total 40 columns):
resultId                  24620 non-null int64
raceId                    24620 non-null int64
driverId                  24620 non-null int64
constructorId             24620 non-null int64
number_result             24620 non-null object
grid_result               24620 non-null int64
positionText_result       24620 non-null object
positionOrder_result      24620 non-null int64
points_result             24620 non-null float64
laps_result               24620 non-null int64
time_result               24620 non-null object
fastestLap_result         24620 non-null object
rank_result               24620 non-null object
fastestLapTime_result     24620 non-null object
fastestLapSpeed_result    24620 non-null object
statusId                  24620 non-null int64
driverRef_driver          24620 non-null object
number_driver             24620 non-null object
code_driver               24620 no

In [18]:
df_all.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number_result',
       'grid_result', 'positionText_result', 'positionOrder_result',
       'points_result', 'laps_result', 'time_result', 'fastestLap_result',
       'rank_result', 'fastestLapTime_result', 'fastestLapSpeed_result',
       'statusId', 'driverRef_driver', 'number_driver', 'code_driver',
       'forename_driver', 'surname_driver', 'dob_driver', 'nationality_driver',
       'year_race', 'round_race', 'circuitId', 'name_race', 'date_race',
       'time_race', 'circuitRef_cir', 'name_cir', 'location_cir',
       'country_cir', 'lat_cir', 'lng_cir', 'alt_cir', 'constructorRef_constr',
       'name_constr', 'nationality_constr', 'status'],
      dtype='object')

In [19]:
print(df_all['raceId'].dtype)

int64


### Clean data - convert cols where needed (obj --> int or datetime

In [26]:
# convert cols that should be dates
d_format = '%Y-%m-%d'
df_all['dob_driver'] = df_all['dob_driver'].map(lambda x : dt.datetime.strptime(x, d_format))
df_all['date_race'] = df_all['date_race'].map(lambda x : dt.datetime.strptime(x, d_format))

#time
#df_all['time_race'] = df_all['time_race'].map(lambda x : dt.datetime.strptime(x, '%H:%M:%S'))

#timings - timedelta objs
'''
df_all['time_result'] = df_all['time_result'].map(lambda x : pd.to_timedelta(x, unit=?))
fastestLapTime_result
'''

TypeError: strptime() argument 1 must be str, not Timestamp

In [21]:
# convert cols that should be int
df_all.astype({'number_result' : 'int32', 'fastestLapSpeed_result' : 'int32', 'fastestLap_result':'int32', 'fastestLapSpeed_result':'float32'})

,resultId,raceId,driverId,constructorId,number_result,grid_result,positionText_result,positionOrder_result,points_result,laps_result,...,name_cir,location_cir,country_cir,lat_cir,lng_cir,alt_cir,constructorRef_constr,name_constr,nationality_constr,status
0,1,18,1,1,22,1,1,1,10.0,58,...,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.9680,10,mclaren,McLaren,British,Finished
1,2,18,2,2,3,5,2,2,8.0,58,...,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.9680,10,bmw_sauber,BMW Sauber,German,Finished
2,3,18,3,3,7,7,3,3,6.0,58,...,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.9680,10,williams,Williams,British,Finished
3,4,18,4,4,5,11,4,4,5.0,58,...,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.9680,10,renault,Renault,French,Finished
4,5,18,5,1,23,3,5,5,4.0,58,...,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.9680,10,mclaren,McLaren,British,Finished
5,6,18,6,3,8,13,6,6,3.0,57,...,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.9680,10,williams,Williams,British,+1 Lap
6,7,18,7,5,14,17,7,7,2.0,55,...,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.9680,10,toro_rosso,Toro Rosso,Italian,Engine
7,8,18,8,6,1,15,8,8,1.0,53,...,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.9680,10,ferrari,Ferrari,Italian,Engine
8,9,18,9,2,4,2,R,9,0.0,47,...,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.9680,10,bmw_sauber,BMW Sauber,German,Collision
9,10,18,10,7,12,18,R,10,0.0,43,...,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.9680,10,toyota,Toyota,Japanese,Accident


In [22]:
#check nationality and other categorical cols don't have misspelling - check for uniques to do this

In [23]:
#ORDER - mask - grouby - agg
#Returns a DataFrame since 
mask_wins = df_all['positionOrder'] == 1
mask_grp_agg = df_all[(mask_wins)].groupby(['name']).sum()
mask_grp_agg.head(10)

KeyError: 'positionOrder'

In [24]:
mask_grp_agg_mult_col = df_all[(mask_wins)].groupby(['name']).sum()[['positionOrder', 'points']]
mask_grp_agg_mult_col.head(10)

NameError: name 'mask_wins' is not defined

In [ ]:
#interesting, this one works but the next cell doesn't
mask_grp_agg_mult_col.sort_values(by=['positionOrder'], ascending=False).head(10)

In [ ]:
# huh, it ended up working here... for fun changed sort col
mask_grp_agg_mult_col_sort = df_all[(mask_wins)].groupby(['name']).sum()[['positionOrder', 'points']].sort_values(by=['points'], ascending=False).head(10)
mask_grp_agg_mult_col_sort

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist()